In [1]:
import os
import shutil
import math
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.models import Sequential
from keras._tf_keras.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras._tf_keras.keras.optimizers import Adam

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d rm1000/brain-tumor-mri-scans

Dataset URL: https://www.kaggle.com/datasets/rm1000/brain-tumor-mri-scans
License(s): CC0-1.0
 98% 232M/236M [00:02<00:00, 99.4MB/s]
100% 236M/236M [00:02<00:00, 91.9MB/s]


In [4]:
!unzip brain-tumor-mri-scans.zip

Streaming output truncated to the last 5000 lines.
  inflating: healthy/0402.jpg        
  inflating: healthy/0403.jpg        
  inflating: healthy/0404.jpg        
  inflating: healthy/0405.jpg        
  inflating: healthy/0406.jpg        
  inflating: healthy/0407.jpg        
  inflating: healthy/0408.jpg        
  inflating: healthy/0409.jpg        
  inflating: healthy/0410.jpg        
  inflating: healthy/0411.jpg        
  inflating: healthy/0412.jpg        
  inflating: healthy/0413.jpg        
  inflating: healthy/0414.jpg        
  inflating: healthy/0415.jpg        
  inflating: healthy/0416.jpg        
  inflating: healthy/0417.jpg        
  inflating: healthy/0418.jpg        
  inflating: healthy/0419.jpg        
  inflating: healthy/0420.jpg        
  inflating: healthy/0421.jpg        
  inflating: healthy/0422.jpg        
  inflating: healthy/0423.jpg        
  inflating: healthy/0424.jpg        
  inflating: healthy/0425.jpg        
  inflating: healthy/0426.jpg        

In [5]:
!rm -r sample_data

In [6]:
healthy_dir = '/content/healthy'
meningioma_dir = '/content/meningioma'
pituitary_dir = '/content/pituitary'
glioma_dir = '/content/glioma'

train_dir = '/content/train'
validate_dir = '/content/validate'
test_dir = '/content/test'

os.makedirs(os.path.join(train_dir, 'healthy'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'meningioma'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'pituitary'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'glioma'), exist_ok=True)

os.makedirs(os.path.join(validate_dir, 'healthy'), exist_ok=True)
os.makedirs(os.path.join(validate_dir, 'meningioma'), exist_ok=True)
os.makedirs(os.path.join(validate_dir, 'pituitary'), exist_ok=True)
os.makedirs(os.path.join(validate_dir, 'glioma'), exist_ok=True)

os.makedirs(os.path.join(test_dir, 'healthy'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'meningioma'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'pituitary'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'glioma'), exist_ok=True)

def split_data(source_dir, train_dir, validate_dir, test_dir, train_split=0.70, validate_split=0.15):
    files = os.listdir(source_dir)
    np.random.shuffle(files)

    train_size = int(len(files) * train_split)
    validate_size = int(len(files)* validate_split)

    train_files = files[:train_size]
    validate_files = files[train_size:train_size + validate_size]
    test_files = files[train_size + validate_size:]

    for file in train_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(train_dir, file))

    for file in validate_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(validate_dir, file))

    for file in test_files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(test_dir, file))


split_data(healthy_dir, os.path.join(train_dir, 'healthy'),
           os.path.join(validate_dir, 'healthy'),
           os.path.join(test_dir, 'healthy'))

split_data(meningioma_dir, os.path.join(train_dir, 'meningioma'),
           os.path.join(validate_dir, 'meningioma'),
           os.path.join(test_dir, 'meningioma'))

split_data(pituitary_dir, os.path.join(train_dir, 'pituitary'),
           os.path.join(validate_dir, 'pituitary'),
           os.path.join(test_dir, 'pituitary'))

split_data(glioma_dir, os.path.join(train_dir, 'glioma'),
           os.path.join(validate_dir, 'glioma'),
           os.path.join(test_dir, 'glioma'))

print("Data has been split into training, validation, and testing datasets.")


Data has been split into training, validation, and testing datasets.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validate_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(75, 75), batch_size=32, class_mode='categorical')

validate_generator = validate_datagen.flow_from_directory(
    validate_dir, target_size=(75, 75), batch_size=32, class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(75, 75), batch_size=32, class_mode='categorical', shuffle=False)

# Display class labels
print(f"Class labels: {train_generator.class_indices}")

In [8]:
class_indices = train_generator.class_indices
class_labels = list(class_indices.keys())
class_counts = train_generator.classes.sum(axis=0)

print(f"Class distribution:")
for i, label in enumerate(class_labels):
    print(f"{label}: {np.sum(train_generator.classes == i)}")

Class distribution:
glioma: 1134
healthy: 1400
meningioma: 1151
pituitary: 1229


In [9]:
class_indices = validate_generator.class_indices
class_labels = list(class_indices.keys())
class_counts = validate_generator.classes.sum(axis=0)

print(f"Distribution:")
for i, label in enumerate(class_labels):
    print(f"{label}: {np.sum(validate_generator.classes == i)}")

Class distribution:
glioma: 243
healthy: 300
meningioma: 246
pituitary: 263


In [10]:
class_indices = test_generator.class_indices
class_labels = list(class_indices.keys())
class_counts = test_generator.classes.sum(axis=0)

print(f"Distribution:")
for i, label in enumerate(class_labels):
    print(f"{label}: {np.sum(test_generator.classes == i)}")

Class distribution:
glioma: 244
healthy: 300
meningioma: 248
pituitary: 265


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(75, 75, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validate_generator,
    validation_steps=len(validate_generator)
)



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 73, 73, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 34, 34, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 17, 17, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 15, 15, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 896,708 (3.42 MB)

 Trainable params: 896,708 (3.42 MB)

 Non-trainable params: 0 (0.00 B)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 64s 396ms/step - accuracy: 0.4443 - loss: 1.1698 - val_accuracy: 0.7310 - val_loss: 0.7219
Epoch 2/20
145/154 ━━━━━━━━━━━━━━━━━━━━ 3s 386ms/step - accuracy: 0.7209 - loss: 0.7152

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save('tumor_classification.keras')